test_Mistral-7B-Instruct.ipynbと同様。"NousResearch/Nous-Capybara-34B"と"mistralai/Mixtral-8x7B-Instruct-v0.1"を40サンプルで検証

In [1]:
from h2ogpte import H2OGPTE

import pandas as pd
from sklearn.metrics import f1_score, accuracy_score

from tqdm import tqdm

In [2]:
f = open('tmp/h2ogpte_key_hatespeechdetect.txt', 'r')
API_KEY = f.read()
f.close()

In [3]:
client = H2OGPTE(
    address="https://playground.h2ogpte.h2o.ai",
    api_key=API_KEY)
client

Please install the correct version of H2OGPTE with `pip install h2ogpte==1.5.0-dev6`.
You can enable strict version checking by passing strict_version_check=True.


In [4]:
# LLM一覧
models = client.get_llms()
[m['base_model'] for m in models]

['mistralai/Mixtral-8x7B-Instruct-v0.1',
 'h2oai/h2ogpt-4096-llama2-70b-chat',
 'NousResearch/Nous-Capybara-34B',
 'mistralai/Mistral-7B-Instruct-v0.2',
 'h2oai/h2o-danube2-1.8b-chat',
 'SeaLLMs/SeaLLM-7B-v2.5',
 'mistral-small-latest',
 'mistral-large-latest',
 'mistral-medium',
 'claude-3-sonnet-20240229',
 'claude-3-opus-20240229',
 'claude-3-haiku-20240307',
 'gemini-1.5-pro-latest',
 'gpt-3.5-turbo-0613',
 'gpt-3.5-turbo-16k-0613',
 'gpt-35-turbo-1106',
 'gpt-4-1106-preview']

In [5]:
chat_session_id = client.create_chat_session_on_default_collection()
chat_session_id

'0dd929b8-a13d-40e4-a2ee-377987abad50'

In [7]:
MODEL = "NousResearch/Nous-Capybara-34B"
SYS_PROMPT = "あなたは文章がヘイトスピーチかどうかを判定するAIです。質問には必ずYes(ヘイトスピーチ)かNo(ヘイトスピーチでない)で答えます。判定の理由は述べる必要はありません。"

In [8]:
text = '''"稲田、今井、豊田、昭恵とあんだけ女に足引っ張られたら
女性活躍なんて言ってられんよなｗ
減らされて当然なのをわからないから女は余計にダメなんだよ。"！'''

#text = '''"ま、バイト（フリーター)でもいいんじゃないの？
#後悔しないのなら"'''
text

'"稲田、今井、豊田、昭恵とあんだけ女に足引っ張られたら\n女性活躍なんて言ってられんよなｗ\n減らされて当然なのをわからないから女は余計にダメなんだよ。"！'

In [9]:
with client.connect(chat_session_id) as session:
    answer = session.query(
        message = text,
        system_prompt=SYS_PROMPT,
        llm=MODEL,
        llm_args={"max_new_tokens":256},
        rag_config={"rag_type": "llm_only",},
    ).content
    print(answer)

Yes.


In [10]:
!ls tmp

h2ogpte-1.5.0.dev2-py3-none-any.whl h2ogpte_key_hatespeechdetect.txt
h2ogpte_key.txt                     hatespeech_test_40sample.csv


In [14]:
df_sampe = pd.read_csv('tmp/hatespeech_test_40sample.csv')
print(df_sampe.shape)
df_sampe.head()

(40, 5)


,id,source,text,label,hatespeech
0,2c37f3740,newsplus,芸スポにも人来とくれよ\nスポーツなのに人少ないんすよ,0,False
1,58a409435,newsplus,12万人のデモが何故か独り歩きして世界をめぐる\n慰安婦報道みたいな様相を呈してきた,0,False
2,0ab60cba6,news4vip,年配の方だと、高いホテルでディナーやサービスを楽しむっていう旅行があるんだよ,0,False
3,73ab69b4e,news4vip,税金バカ食いしてんのはニートじゃなくて障害者とナマポだろ\nそんな事も分からんほどアホなのか,1,False
4,ed881a220,newsplus,パヨクっつーか、やべー奴のツイやインスタのイメージ\n\nツイでこのタグの量はだいたい面倒くさいの,0,False


In [26]:
df_sampe = df_sampe.rename(columns={'hatespeech':'res_Mistral-7B-Instruct-v02'})

In [15]:
def get_judgements(text_list):
    answers = []
    with client.connect(chat_session_id) as session:
        for text in tqdm(text_list):
            answer = session.query(
                message = text,
                system_prompt=SYS_PROMPT,
                llm=MODEL,
                rag_config={"rag_type": "llm_only",},
            ).content
            answers.append(answer)
    return answers

## NousResearch/Nous-Capybara-34B
https://huggingface.co/NousResearch/Nous-Capybara-34B

In [16]:
MODEL = "NousResearch/Nous-Capybara-34B"
SYS_PROMPT = "あなたは文章がヘイトスピーチかどうかを判定するAIです。質問には必ずYes(ヘイトスピーチ)かNo(ヘイトスピーチでない)で答えます。判定の理由は述べる必要はありません。"

outputs = get_judgements(df_sampe['text'])

100%|██████████| 40/40 [01:50<00:00,  2.76s/it]


In [17]:
len(outputs)

40

In [18]:
outputs

['No.',
 'No.',
 'No.',
 'Yes.',
 'No.',
 'No.',
 'Yes.',
 'Yes.',
 'Yes.',
 'No.',
 'No.',
 'Yes.',
 'Yes.',
 'No.',
 'No.',
 'Yes.',
 'Yes.',
 'Yes.',
 'No.',
 'Yes.',
 'No.',
 'No.',
 'Yes.',
 'Yes.',
 'No.',
 'Yes.',
 'No.',
 'No.',
 'No.',
 'No.',
 'Yes.',
 'No.',
 'Yes.',
 'Yes.',
 'Yes.',
 'Yes.',
 'No.',
 'Yes.',
 'Yes.',
 'Yes.']

In [21]:
judges = []
for i, txt in enumerate(outputs):
    firstpart = txt[:15]    # 文字列の先頭
    #print(i, ' : ' ,firstpart)
    judge = 'yes' in firstpart.lower()   # 先頭にYes/yesが含まれるかどうか
    #print('Hate Speech? ', judge)
    judges.append(judge)

In [27]:
df_sampe['res_Nous-Capybara-34B'] = judges

In [29]:
df_sampe.head()

,id,source,text,label,res_Mistral-7B-Instruct-v02,res_Nous-Capybara-34B
0,2c37f3740,newsplus,芸スポにも人来とくれよ\nスポーツなのに人少ないんすよ,0,False,False
1,58a409435,newsplus,12万人のデモが何故か独り歩きして世界をめぐる\n慰安婦報道みたいな様相を呈してきた,0,False,False
2,0ab60cba6,news4vip,年配の方だと、高いホテルでディナーやサービスを楽しむっていう旅行があるんだよ,0,False,False
3,73ab69b4e,news4vip,税金バカ食いしてんのはニートじゃなくて障害者とナマポだろ\nそんな事も分からんほどアホなのか,1,False,True
4,ed881a220,newsplus,パヨクっつーか、やべー奴のツイやインスタのイメージ\n\nツイでこのタグの量はだいたい面倒くさいの,0,False,False


In [30]:
df_sampe['res_Nous-Capybara-34B'].value_counts()

res_Nous-Capybara-34B
True     21
False    19
Name: count, dtype: int64

In [31]:
pd.crosstab(df_sampe['label'], df_sampe['res_Nous-Capybara-34B'], margins=True)

res_Nous-Capybara-34B,False,True,All
label,,,
0,14,6,20
1,5,15,20
All,19,21,40


In [32]:
f1_score(y_true=df_sampe['label'], y_pred=df_sampe['res_Nous-Capybara-34B'])

0.7317073170731707

In [33]:
accuracy_score(y_true=df_sampe['label'], y_pred=df_sampe['res_Nous-Capybara-34B'])

0.725

## mistralai/Mixtral-8x7B-Instruct-v0.1
https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1

In [34]:
MODEL = "mistralai/Mixtral-8x7B-Instruct-v0.1"
SYS_PROMPT = "あなたは文章がヘイトスピーチかどうかを判定するAIです。質問には必ずYes(ヘイトスピーチ)かNo(ヘイトスピーチでない)で答えます。判定の理由は述べる必要はありません。"

outputs = get_judgements(df_sampe['text'])

100%|██████████| 40/40 [01:58<00:00,  2.96s/it]


In [35]:
len(outputs)

40

In [36]:
outputs

['No(ヘイトスピーチでない)',
 'No(ヘイトスピーチでない)',
 'No(ヘイトスピーチでない)',
 'Yes',
 'Yes (ヘイトスピーチ)',
 'Yes',
 'Yes',
 'No (ヘイトスピーチでない)',
 'No (ヘイトスピーチでない)',
 'Yes(ヘイトスピーチ)',
 'No (ヘイトスピーチでない)',
 'Yes',
 'Yes',
 'No(ヘイトスピーチでない)',
 'Yes, this is a hate speech. The message contains a derogatory and dismissive tone towards the person being addressed.',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes\n\nThe first sentence is a threat to physically harm someone, and the second sentence is derogatory and dehumanizing towards the person referred to as "ファビョ太君". Both sentences are examples of hate speech.',
 'Yes',
 'Yes',
 'Yes (ヘイトスピーチ)',
 'Yes',
 'Yes',
 'No.\n\n<details>\n<summary>判定理由</summary>\n\nこの文章は、在日（在日韓国人）に対する偏見や差別を示唆するものではありません。単に在日韓国人についての質問であるため、ヘイトスピーチではありません。\n</details>',
 'No(ヘイトスピーチでない)',
 'No (ヘイトスピーチでない)',
 'Yes (ヘイトスピーチ)',
 'No(ヘイトスピーチでない)',
 'Yes\n\n<h3>判定理由</h3>\n<p>これらの文章は、男性を一般化して差別的に扱っているため、ヘイトスピーチであると判定しました。</p>\n<p>特に、「女性が真に自立した社会を築くためには、男共を皆殺しにしなければいけない」という文脈は、男性を無条件に殺すことを主張しているため、非常に過激で差別的で

In [37]:
judges = []
for i, txt in enumerate(outputs):
    firstpart = txt[:15]    # 文字列の先頭
    #print(i, ' : ' ,firstpart)
    judge = 'yes' in firstpart.lower()   # 先頭にYes/yesが含まれるかどうか
    #print('Hate Speech? ', judge)
    judges.append(judge)

In [38]:
df_sampe['res_Mixtral-8x7B-Instruct-v01'] = judges
df_sampe.head()

,id,source,text,label,res_Mistral-7B-Instruct-v02,res_Nous-Capybara-34B,res_Mixtral-8x7B-Instruct-v01
0,2c37f3740,newsplus,芸スポにも人来とくれよ\nスポーツなのに人少ないんすよ,0,False,False,False
1,58a409435,newsplus,12万人のデモが何故か独り歩きして世界をめぐる\n慰安婦報道みたいな様相を呈してきた,0,False,False,False
2,0ab60cba6,news4vip,年配の方だと、高いホテルでディナーやサービスを楽しむっていう旅行があるんだよ,0,False,False,False
3,73ab69b4e,news4vip,税金バカ食いしてんのはニートじゃなくて障害者とナマポだろ\nそんな事も分からんほどアホなのか,1,False,True,True
4,ed881a220,newsplus,パヨクっつーか、やべー奴のツイやインスタのイメージ\n\nツイでこのタグの量はだいたい面倒くさいの,0,False,False,True


In [39]:
df_sampe['res_Mixtral-8x7B-Instruct-v01'].value_counts()

res_Mixtral-8x7B-Instruct-v01
True     26
False    14
Name: count, dtype: int64

In [40]:
pd.crosstab(df_sampe['label'], df_sampe['res_Mixtral-8x7B-Instruct-v01'], margins=True)

res_Mixtral-8x7B-Instruct-v01,False,True,All
label,,,
0,13,7,20
1,1,19,20
All,14,26,40


In [41]:
f1_score(y_true=df_sampe['label'], y_pred=df_sampe['res_Mixtral-8x7B-Instruct-v01'])

0.8260869565217391

In [42]:
accuracy_score(y_true=df_sampe['label'], y_pred=df_sampe['res_Mixtral-8x7B-Instruct-v01'])

0.8

In [43]:
df_sampe.to_csv('hatespeech_test_40sample_RES.csv', index=False)